# TEST KOLMOGOROV SMIRNOV

In [1]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Wed Sep 17 17:38:27 2025"

In [2]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,656852,35.1,1439374,76.9,1439374,76.9
Vcells,1224762,9.4,8388608,64.0,1924961,14.7


In [3]:
require("data.table")

if( !require("R.utils")) install.packages("R.utils")
require("R.utils")

Loading required package: data.table

Loading required package: R.utils

Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.2 (2022-06-13 22:00:14 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.27.1 (2025-05-02 21:00:05 UTC) successfully loaded. See ?R.oo for help.


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

    throw


The following objects are masked from ‘package:methods’:

    getClasses, getMethods


The following objects are masked from ‘package:base’:

    attach, detach, load, save


R.utils v2.13.0 (2025-02-24 21:20:02 UTC) successfully loaded. See ?R.utils for help.


Attaching package: ‘R.utils’


The following object is masked from ‘package:utils’:

    timestamp


The following objects are masked from ‘package:base’:

    cat, commandArgs, getOption, isOpen, nullfile, parse, use, warnings




In [4]:
PARAM <- list()
PARAM$semilla_primigenia <- 102191

PARAM$experimento <- 1999
PARAM$dataset <- "analistajr_competencia_2025.csv.gz"

#### Carpeta del Experimento

In [5]:
# carpeta de trabajo

setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("KS", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

### 9.7.1   Preprocesamiento del dataset

#### 9.7.1.1  DT incorporar dataset

In [6]:
# lectura del dataset
dataset <- fread(paste0("/content/datasets/", PARAM$dataset))

In [7]:
View(dataset)


numero_de_cliente,foto_mes,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,⋯,Visa_mfinanciacion_limite,Visa_Fvencimiento,Visa_msaldototal,Visa_mlimitecompra,Visa_fultimo_cierre,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_mpagominimo,clase_ternaria
<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<int>,<dbl>,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>,<chr>
29187730,201901,1,62,296,14301.42,43997.41,8970.82,-1262.22,5806.64,⋯,412896.5,-1337,127146.16,458783.6,22,5604,122057.93,50,11190.42,CONTINUA
29187961,201901,1,59,296,7542.75,34281.62,1960.95,-46.22,4804.79,⋯,412395.4,-1915,47726.51,458226.8,1,7604,48761.71,41,2087.94,CONTINUA
29193101,201901,1,66,349,5749.83,29663.24,4745.47,-3592.97,4449.20,⋯,207286.7,-1763,62541.14,230318.5,8,1540,26574.05,22,5841.54,CONTINUA
29193281,201901,1,52,142,12742.79,43614.97,12275.05,-718.19,1116.94,⋯,38005.2,-1154,17241.41,42228.0,1,2864,19809.29,8,1032.24,CONTINUA
29198891,201901,1,53,296,4237.90,23234.30,1176.26,-680.91,3291.01,⋯,50673.6,-211,22805.49,56304.0,1,6756,26750.83,4,1067.43,CONTINUA
29200770,201901,1,56,246,12527.87,60467.54,8240.80,281.61,3373.18,⋯,416459.8,-1763,8826.08,462742.9,8,4474,5590.22,5,1736.04,CONTINUA
29207790,201901,1,40,46,728.57,13014.84,219.22,-89.35,523.58,⋯,207286.7,-1549,3428.47,230318.5,8,1383,3079.74,7,304.98,CONTINUA
29207901,201901,0,81,296,4054.47,31669.65,1172.10,-131.76,2589.02,⋯,537595.4,-1154,2619.67,597328.2,1,7994,1506.68,2,0.00,CONTINUA
29215330,201901,1,48,237,3002.34,41679.79,920.64,-2332.47,4107.15,⋯,207286.7,-1215,8162.11,230318.5,8,7217,9100.89,7,480.93,CONTINUA


#### 9.3.1.3  DR  Data Drifting
Se intenta corregir el data drifting, ajustando por algunos indices financieros

In [8]:
# meses que me interesan para el ajuste de variables monetarias
vfoto_mes <- c(
  201901, 201902, 201903, 201904, 201905, 201906,
  201907, 201908, 201909, 201910, 201911, 201912,
  202001, 202002, 202003, 202004, 202005, 202006,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202104, 202105, 202106,
  202107, 202108, 202109
)


In [9]:
# los valores que siguen fueron calculados por alumnos

# momento 1.0  Sep-2021
vIPC <- c(2.787281901122360, 2.686132306921320, 2.566052833332770,
  2.480603534259520, 2.406972565811860, 2.343282033546600,
  2.292887450432550, 2.205675360182500, 2.083072734166780,
  2.016654576841490, 1.934356651408890, 1.864553234816590,
  1.823472319453580, 1.787479114947270, 1.729648848066530,
  1.704145079892160, 1.678254046331520, 1.641427967120310,
  1.610281979296860, 1.567945714631560, 1.524712170969590,
  1.469446722849830, 1.424438664000510, 1.369579245086460,
  1.316282576323060, 1.270854218792650, 1.212522114036500,
  1.164987215923810, 1.127519974398630, 1.092827636775390,
  1.061028266084610, 1.035467879533110, 1.000000000000000)

vdolar_blue <- c(
   39.045455,  38.402500,  41.639474,
   44.274737,  46.095455,  45.063333,
   43.983333,  54.842857,  61.059524,
   65.545455,  66.750000,  72.368421,
   77.477273,  78.191667,  82.434211,
  101.087500, 126.236842, 125.857143,
  130.782609, 133.400000, 137.954545,
  170.619048, 160.400000, 153.052632,
  157.900000, 149.380952, 143.615385,
  146.250000, 153.550000, 162.000000,
  178.478261, 180.878788, 184.357143
)

vdolar_oficial <- c(
   38.430000,  39.428000,  42.542105,
   44.354211,  46.088636,  44.955000,
   43.751429,  54.650476,  58.790000,
   61.403182,  63.012632,  63.011579,
   62.983636,  63.580556,  65.200000,
   67.872000,  70.047895,  72.520952,
   75.324286,  77.488500,  79.430909,
   83.134762,  85.484737,  88.181667,
   91.474000,  93.997778,  96.635909,
   98.526000,  99.613158, 100.619048,
  101.619048, 102.569048, 103.781818
)

vNucleo <- c(2.9509401776, 2.8413904076, 2.7169681247,
  2.6183338075, 2.5376976850, 2.4702213038,
  2.4182843036, 2.3110987598, 2.1713146424,
  2.0917312214, 2.0117031138, 1.9402570077,
  1.8940250995, 1.8499633664, 1.7939805354,
  1.7637193990, 1.7358472741, 1.6965692402,
  1.6546523624, 1.6059398018, 1.5695744147,
  1.5170891738, 1.4596143906, 1.3918188786,
  1.3393707547, 1.2869937330, 1.2312659603,
  1.1775806320, 1.1379817057, 1.0983355780,
  1.0649648892, 1.0326212556, 1.0000000000)


In [10]:
tb_indices <- as.data.table( list(
  "IPC" = vIPC,
  "dolar_blue" = vdolar_blue,
  "dolar_oficial" = vdolar_oficial,
  "Nucleo" = vNucleo
  )
)

tb_indices[[ 'foto_mes' ]] <- vfoto_mes

tb_indices

IPC,dolar_blue,dolar_oficial,Nucleo,foto_mes
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2.787282,39.04545,38.43000,2.950940,201901
2.686132,38.40250,39.42800,2.841390,201902
2.566053,41.63947,42.54210,2.716968,201903
2.480604,44.27474,44.35421,2.618334,201904
2.406973,46.09546,46.08864,2.537698,201905
2.343282,45.06333,44.95500,2.470221,201906
2.292887,43.98333,43.75143,2.418284,201907
2.205675,54.84286,54.65048,2.311099,201908
2.083073,61.05952,58.79000,2.171315,201909


In [11]:
drift_Nucleo <- function(campos_monetarios) {
  cat( "inicio drift_Nucleo()\n")

  dataset[tb_indices,
    on = c("foto_mes"),
    (campos_monetarios) := .SD * i.Nucleo,
    .SDcols = campos_monetarios
  ]

  cat( "fin drift_Nucleo()\n")
}


In [12]:
drift_dolar_oficial <- function(campos_monetarios) {
  cat( "inicio drift_dolar_oficial()\n")

  dataset[tb_indices,
    on = c("foto_mes"),
    (campos_monetarios) := .SD / i.dolar_oficial,
    .SDcols = campos_monetarios
  ]

  cat( "fin drift_dolar_oficial()\n")
}


In [13]:
drift_dolar_blue <- function(campos_monetarios) {
  cat( "inicio drift_dolar_blue()\n")

  dataset[tb_indices,
    on = c("foto_mes"),
    (campos_monetarios) := .SD / i.dolar_blue,
    .SDcols = campos_monetarios
  ]

  cat( "fin drift_dolar_blue()\n")
}


In [14]:
drift_IPC <- function(campos_monetarios) {
  cat( "inicio drift_IPC()\n")

  dataset[tb_indices,
    on = c("foto_mes"),
    (campos_monetarios) := .SD * i.IPC,
    .SDcols = campos_monetarios
  ]

  cat( "fin drift_IPC()\n")
}


In [15]:
drift_rank_simple <- function(campos_drift) {

  cat( "inicio drift_rank_simple()\n")
  for (campo in campos_drift)
  {
    cat(campo, " ")
    dataset[, paste0(campo, "_rank") :=
      (frank(get(campo), ties.method = "random") - 1) / (.N - 1), by = list(foto_mes)]
    dataset[, (campo) := NULL]
  }
  cat( "fin drift_rank_simple()\n")
}


In [16]:
# El cero se transforma en cero
# los positivos se rankean por su lado
# los negativos se rankean por su lado

drift_rank_cero_fijo <- function(campos_drift) {

  cat( "inicio drift_rank_cero_fijo()\n")
  for (campo in campos_drift)
  {
    cat(campo, " ")
    dataset[get(campo) == 0, paste0(campo, "_rank") := 0]
    dataset[get(campo) > 0, paste0(campo, "_rank") :=
      frank(get(campo), ties.method = "random") / .N, by = list(foto_mes)]

    dataset[get(campo) < 0, paste0(campo, "_rank") :=
      -frank(-get(campo), ties.method = "random") / .N, by = list(foto_mes)]
    dataset[, (campo) := NULL]
  }
  cat("\n")
  cat( "fin drift_rank_cero_fijo()\n")
}


In [17]:
drift_estandarizar <- function(campos_drift) {
  cat("inicio drift_estandarizar()\n")
  for (campo in campos_drift) {
    cat(campo, " ")
    
    # Check if column exists
    if (!campo %in% names(dataset)) {
      warning(paste("Column", campo, "not found in dataset"))
      next
    }
    
    # Standardize by foto_mes groups
    dataset[, paste0(campo, "_normal") := {
      valores <- get(campo)
      media <- mean(valores, na.rm = TRUE)
      desv <- sd(valores, na.rm = TRUE)
      
      # Handle cases where sd = 0 (constant values)
      if (is.na(desv) || desv == 0) {
        rep(0, length(valores))  # or rep(NA, length(valores))
      } else {
        (valores - media) / desv
      }
    }, by = foto_mes]
    
    # Remove original column
    dataset[, (campo) := NULL]
  }
  cat("fin drift_estandarizar()\n")
}

In [18]:
# por como armé los nombres de campos,
#  estos son los campos que expresan variables monetarias
campos_monetarios <- colnames(dataset)
campos_monetarios <- campos_monetarios[campos_monetarios %like%
  "^(m|Visa_m|Master_m|vm_m)"]

campos_monetarios

[1] "mrentabilidad"                      "mrentabilidad_annual"              
 [3] "mcomisiones"                        "mactivos_margen"                   
 [5] "mpasivos_margen"                    "mcuenta_corriente"                 
 [7] "mcaja_ahorro"                       "mcuentas_saldo"                    
 [9] "mtarjeta_visa_consumo"              "mtarjeta_master_consumo"           
[11] "mprestamos_personales"              "mpayroll"                          
[13] "mttarjeta_visa_debitos_automaticos" "mcomisiones_mantenimiento"         
[15] "mtransferencias_recibidas"          "Master_mfinanciacion_limite"       
[17] "Master_msaldototal"                 "Master_mlimitecompra"              
[19] "Master_mconsumototal"               "Master_mpagominimo"                
[21] "Visa_mfinanciacion_limite"          "Visa_msaldototal"                  
[23] "Visa_mlimitecompra"                 "Visa_mconsumototal"                
[25] "Visa_mpagominimo"

In [19]:
# ejecuto el Data Drifting
setorder( dataset, numero_de_cliente, foto_mes )

metodoDrift = "rank_simple"

PARAM$DR$metodo <- metodoDrift

switch(PARAM$DR$metodo,
  "ninguno"        = cat("No hay correccion del data drifting"),
  "rank_simple"    = drift_rank_simple(campos_monetarios),
  "rank_cero_fijo" = drift_rank_cero_fijo(campos_monetarios),
  "IPC"      = drift_IPC(campos_monetarios),
  "dolar_blue"     = drift_dolar_blue(campos_monetarios),
  "dolar_oficial"  = drift_dolar_oficial(campos_monetarios),
  "Nucleo"            = drift_Nucleo(campos_monetarios),
  "estandarizar"   = drift_estandarizar(campos_monetarios)
)


inicio drift_rank_simple()
mrentabilidad  mrentabilidad_annual  mcomisiones  mactivos_margen  mpasivos_margen  mcuenta_corriente  mcaja_ahorro  mcuentas_saldo  mtarjeta_visa_consumo  mtarjeta_master_consumo  mprestamos_personales  mpayroll  mttarjeta_visa_debitos_automaticos  mcomisiones_mantenimiento  mtransferencias_recibidas  Master_mfinanciacion_limite  Master_msaldototal  Master_mlimitecompra  Master_mconsumototal  Master_mpagominimo  Visa_mfinanciacion_limite  Visa_msaldototal  Visa_mlimitecompra  Visa_mconsumototal  Visa_mpagominimo  fin drift_rank_simple()


In [23]:
View(dataset)

numero_de_cliente,foto_mes,internet,cliente_edad,cliente_antiguedad,cproductos,cdescubierto_preacordado,ctarjeta_visa,ctarjeta_visa_transacciones,ctarjeta_master,⋯,Master_mfinanciacion_limite_rank,Master_msaldototal_rank,Master_mlimitecompra_rank,Master_mconsumototal_rank,Master_mpagominimo_rank,Visa_mfinanciacion_limite_rank,Visa_msaldototal_rank,Visa_mlimitecompra_rank,Visa_mconsumototal_rank,Visa_mpagominimo_rank
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
29186441,202003,1,61,280,7,1,1,8,1,⋯,0.2284666,0.14788875,0.2266483,0.978382383,0.02821739,0.1629066,0.4380430,0.1615260,0.5469392,0.38261836
29186441,202004,1,61,281,7,1,1,8,1,⋯,0.2158553,0.11691884,0.2289731,0.402992586,0.25121616,0.1522797,0.5110545,0.1699265,0.5783541,0.41231925
29186441,202005,1,61,282,7,1,1,8,1,⋯,0.2097856,0.49219943,0.2255859,0.611763925,0.07508464,0.1471486,0.5822545,0.1760937,0.6252075,0.47878909
29186441,202006,0,61,283,7,1,1,0,1,⋯,0.2190760,0.21377890,0.2254520,0.895334009,0.34826538,0.1735932,0.5050191,0.1810156,0.5381094,0.42834254
29186441,202007,1,61,284,7,1,1,8,1,⋯,0.1991539,0.18672092,0.2109087,0.664664471,0.21035975,0.1689595,0.5487632,0.1433508,0.6119615,0.42895434
29186441,202008,1,61,285,7,1,1,8,1,⋯,0.2240332,0.07299457,0.2143816,0.849312880,0.45190157,0.1482582,0.5322148,0.1562800,0.6061042,0.40977948
29186441,202009,1,61,286,7,1,1,8,1,⋯,0.2217969,0.24024043,0.2210060,0.751439418,0.16662449,0.1562480,0.5516292,0.1448592,0.6087947,0.40683328
29186441,202010,0,61,287,7,1,1,8,1,⋯,0.2089431,0.21868831,0.2100711,0.886535268,0.48074452,0.1535111,0.5482092,0.1572400,0.6249491,0.43455645
29186441,202011,0,61,288,7,1,1,9,1,⋯,0.1987491,0.14628454,0.1985624,0.404997511,0.42397934,0.1484939,0.5252054,0.1414613,0.6175006,0.38010331


head(dataset)

In [21]:
# se intenta corregir el data drifting utilizando algunos indices financieros

In [22]:
# Kolmogorov-Smirnov Test for Data Drift Detection
# Comparing each foto_mes distribution against 202109 as reference

# Load required libraries
require("data.table")

# Parameters - adjust as needed
PARAM <- list()
PARAM$dataset <- ""  # Will be set based on your dataset
PARAM$reference_month <- 202109  # Reference month for comparison
PARAM$alpha <- 0.05  # Significance level

# Check available foto_mes values
foto_mes_values <- sort(unique(dataset$foto_mes))
cat("Available foto_mes values:", paste(foto_mes_values, collapse = ", "), "\n")

# Verify reference month exists
if (!PARAM$reference_month %in% foto_mes_values) {
  stop("Reference month ", PARAM$reference_month, " not found in dataset!")
}

# Get reference data (202109)
reference_data <- dataset[foto_mes == PARAM$reference_month]
cat("Reference month", PARAM$reference_month, "has", nrow(reference_data), "observations\n")

# Get monetary columns for KS testing
# por como armé los nombres de campos,
# estos son los campos que expresan variables monetarias
campos_monetarios <- colnames(dataset)
campos_monetarios <- campos_monetarios[
  campos_monetarios %like% "^(m|Visa_m|Master_m|vm_m)|rank"
]

# Use monetary columns as our test variables
numeric_cols <- campos_monetarios

cat("Testing", length(numeric_cols), "monetary variables\n")
cat("Monetary fields found:", paste(campos_monetarios, collapse = ", "), "\n")
cat("Variables to test:", paste(head(numeric_cols, 10), collapse = ", "), 
    if(length(numeric_cols) > 10) "..." else "", "\n")

# Function to perform KS test for a single variable between two foto_mes
perform_ks_test <- function(var_name, month1_data, month2_data, month1_name, month2_name) {
  # Remove NA values
  var1 <- month1_data[[var_name]]
  var2 <- month2_data[[var_name]]
  
  var1 <- var1[!is.na(var1)]
  var2 <- var2[!is.na(var2)]
  
  if (length(var1) == 0 || length(var2) == 0) {
    return(list(
      variable = var_name,
      comparison = paste(month1_name, "vs", month2_name),
      n1 = length(var1),
      n2 = length(var2),
      ks_statistic = NA,
      p_value = NA,
      significant = NA,
      interpretation = "Insufficient data"
    ))
  }
  
  # Perform KS test (suppress warnings)
  ks_result <- suppressWarnings(ks.test(var1, var2, exact = FALSE))
  
  return(list(
    variable = var_name,
    comparison = paste(month1_name, "vs", month2_name),
    n1 = length(var1),
    n2 = length(var2),
    ks_statistic = round(ks_result$statistic, 4),
    p_value = round(ks_result$p.value, 3),
    significant = ks_result$p.value < PARAM$alpha,
    interpretation = ifelse(ks_result$p.value < PARAM$alpha, 
                           "Significant difference (data drift detected)", 
                           "No significant difference")
  ))
}

# Perform KS tests for each foto_mes against reference
all_results <- list()
result_count <- 0

cat("\n=== PERFORMING KOLMOGOROV-SMIRNOV TESTS ===\n")
cat("method: " ,metodoDrift, "\n")

# Test each foto_mes (except reference) against reference month
test_months <- setdiff(foto_mes_values, PARAM$reference_month)

for (test_month in test_months) {
  cat("\nTesting", test_month, "vs", PARAM$reference_month, "\n")
  
  test_data <- dataset[foto_mes == test_month]
  cat("  Sample size:", nrow(test_data), "observations\n")
  
  # Test each numeric variable
  month_results <- list()
  significant_vars <- 0
  
  for (var_name in numeric_cols) {
    result_count <- result_count + 1
    
    ks_result <- perform_ks_test(var_name, test_data, reference_data, 
                                test_month, PARAM$reference_month)
    
    month_results[[var_name]] <- ks_result
    all_results[[result_count]] <- ks_result
    
    if (!is.na(ks_result$significant) && ks_result$significant) {
      significant_vars <- significant_vars + 1
    }
  }
  
  cat("  Variables with significant drift:", significant_vars, "out of", length(numeric_cols), "\n")
}

# Convert results to data.table for analysis
results_dt <- rbindlist(all_results)

# Summary by foto_mes
cat("\n=== SUMMARY BY FOTO_MES ===\n")
summary_by_month <- results_dt[!is.na(significant), .(
  total_variables = .N,
  significant_drift = sum(significant),
  pct_drift = round(100 * sum(significant) / .N, 1),
  min_p_value = min(p_value, na.rm = TRUE),
  mean_p_value = mean(p_value, na.rm = TRUE),
  max_ks_stat = max(ks_statistic, na.rm = TRUE),
  mean_ks_stat = mean(ks_statistic, na.rm = TRUE)
), by = .(foto_mes = sub(" vs.*", "", comparison))]

print(summary_by_month)

# Variables with most drift across months
cat("\n=== VARIABLES WITH MOST FREQUENT DRIFT ===\n")
drift_by_variable <- results_dt[!is.na(significant), .(
  months_tested = .N,
  months_with_drift = sum(significant),
  pct_months_drift = round(100 * sum(significant) / .N, 1),
  avg_ks_statistic = round(mean(ks_statistic, na.rm = TRUE), 4),
  min_p_value = min(p_value, na.rm = TRUE)
), by = variable][order(-pct_months_drift, -avg_ks_statistic)]

print(head(drift_by_variable, 20))

# Most significant cases (lowest p-values)
cat("\n=== MOST SIGNIFICANT DRIFT CASES ===\n")
most_significant <- results_dt[significant == TRUE][order(p_value)][1:20]

# Format p_value to always show 3 decimals
most_significant_formatted <- most_significant[, .(
  variable, 
  comparison, 
  ks_statistic, 
  p_value = sprintf("%.3f", p_value)
)]
print(most_significant_formatted)

# Export results
output_file <- paste0("ks_test_results", metodoDrift,".csv")
fwrite(summary_by_month, output_file)
cat("\nDetailed results exported to:", output_file, "\n")

# Summary statistics
total_tests <- nrow(results_dt[!is.na(significant)])
significant_tests <- nrow(results_dt[significant == TRUE])

cat("\n=== OVERALL SUMMARY ===\n")
cat("Total KS tests performed:", total_tests, "\n")
cat("Tests showing significant drift:", significant_tests, "\n")
cat("Percentage of tests with drift:", round(100 * significant_tests / total_tests, 1), "%\n")
cat("Reference month:", PARAM$reference_month, "\n")
cat("Significance level:", PARAM$alpha, "\n")

cat("\nInterpretation:")
cat("\n- KS test detects differences in distributions between months")
cat("\n- Significant results (p <", PARAM$alpha, ") indicate data drift")
cat("\n- Higher KS statistics indicate larger distributional differences\n")




Available foto_mes values: 201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908, 201909, 201910, 201911, 201912, 202001, 202002, 202003, 202004, 202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012, 202101, 202102, 202103, 202104, 202105, 202106, 202107, 202108, 202109 
Reference month 202109 has 33094 observations
Testing 25 monetary variables
Monetary fields found: mrentabilidad_rank, mrentabilidad_annual_rank, mcomisiones_rank, mactivos_margen_rank, mpasivos_margen_rank, mcuenta_corriente_rank, mcaja_ahorro_rank, mcuentas_saldo_rank, mtarjeta_visa_consumo_rank, mtarjeta_master_consumo_rank, mprestamos_personales_rank, mpayroll_rank, mttarjeta_visa_debitos_automaticos_rank, mcomisiones_mantenimiento_rank, mtransferencias_recibidas_rank, Master_mfinanciacion_limite_rank, Master_msaldototal_rank, Master_mlimitecompra_rank, Master_mconsumototal_rank, Master_mpagominimo_rank, Visa_mfinanciacion_limite_rank, Visa_msaldototal_rank, Visa_mlimitecompra_rank, Visa_mconsumo